In [2]:
from pydub import AudioSegment
import pandas as pd
meta = pd.read_csv('/Users/andre/Downloads/meta_info.csv')
meta

,index,VoxCeleb1 ID,VGGFace1 ID,Gender,Nationality,Set
0,5,id10006,Abbie_Cornish,f,Australia,dev
1,630,id10631,Ken_Jeong,m,USA,dev
2,633,id10634,Kendra_Wilkinson,f,USA,dev
3,634,id10635,Kenny_Rogers,m,USA,dev
4,635,id10636,Kenton_Duty,m,USA,dev
5,1052,id11053,Scott_Speedman,m,Canada,dev
6,1053,id11054,Sean_Bean,m,UK,dev
7,1110,id11111,Stephen_Fry,m,UK,dev
8,1111,id11112,Stephen_Lang,m,USA,dev
9,1127,id11128,Sung_Kang,m,USA,dev


In [3]:
import os
counter = 1
concatenated_audios = []
for filename in os.listdir('C:/Users/andre/Downloads/data/Data'):
    combined_sounds = AudioSegment.empty()
    for file in os.listdir('C:/Users/andre/Downloads/data/Data/' + filename):
        for audiofile in os.listdir('C:/Users/andre/Downloads/data/Data/' + filename + '/' + file):
            combined_sounds = combined_sounds + AudioSegment.from_wav('/Users/andre/Downloads/data/Data/' + filename + "/" + file + "/" + audiofile)
    combined_sounds.export("/Users/andre/Downloads/Sounds/sound" + str(counter) + ".wav", format="wav")
    concatenated_audios.append(combined_sounds)
    counter = counter + 1

In [25]:
import librosa
import numpy as np
directory = 'C:/Users/andre/Downloads/Sounds'
mel_spectral_matrix = np.array([])

for filename in os.listdir(directory):
    y, sr = librosa.load('C:/Users/andre/Downloads/Sounds/' + filename)
    mel_spectral_matrix = np.append(mel_spectral_matrix, librosa.feature.melspectrogram(y=y, sr=8000, n_fft = int(0.03 * 8000), hop_length = int(0.01 * 8000)))

In [5]:
import librosa
import numpy as np
directory = 'C:/Users/andre/Downloads/Sounds'
mel_spectral_matrix1 = np.array([])

for filename in os.listdir(directory):
    y, sr = librosa.load('C:/Users/andre/Downloads/Sounds/' + filename)
    mel_spectral_matrix1 = np.append(mel_spectral_matrix1, librosa.feature.melspectrogram(y=y[:240000], sr=8000, n_fft = int(0.03 * 8000), hop_length = int(0.01 * 8000)))

In [6]:
mel_spectral_matrix1.shape = (30010, 128)

Here each label is represented by a number (0 - 9) to denote each speaker as speaker0, speaker1,... to speaker9.

In [7]:
output = []
for i in range(10):
    result = []
    result = [i] * 3001
    output = output + result
output = np.array(output)
output.shape = (30010, 1)

# Build a classifier using a Feedforward Neural Networks and Confusion Matrix

I decided to use two optimizers (Adam Optimizer and the Gradient Descent Optimizer) to see which one produced better results for the training accuracy. We see that one produced twice the training accuracy than the other one produced. Therefore, I decided to use the Adam Optimizer from the two options.

In [30]:
import tensorflow as tf
tf.reset_default_graph()
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.model_selection import train_test_split

RANDOM_SEED = 21
tf.set_random_seed(RANDOM_SEED)

def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

def get_data():
    return train_test_split(mel_spectral_matrix1, output, test_size=0.33, random_state=RANDOM_SEED)

def main():
    train_X, test_X, train_y, test_y = get_data()
    # Turn the targets to categorical
    train_y = tf.keras.utils.to_categorical(train_y)
    test_y = tf.keras.utils.to_categorical(test_y)
    # Layer's sizes
    x_size = train_X.shape[1]
    h_size = 256    
    y_size = train_y.shape[1]
    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])
    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.AdamOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(1000):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})
        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: train_X, y: train_y}))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: test_X, y: test_y}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
    
    sess.close()

if __name__ == '__main__':
    main()

Epoch = 1, train accuracy = 16.42%, test accuracy = 16.09%
Epoch = 2, train accuracy = 17.04%, test accuracy = 16.72%
Epoch = 3, train accuracy = 17.61%, test accuracy = 17.18%
Epoch = 4, train accuracy = 18.16%, test accuracy = 17.56%
Epoch = 5, train accuracy = 18.54%, test accuracy = 17.74%
Epoch = 6, train accuracy = 18.74%, test accuracy = 17.93%
Epoch = 7, train accuracy = 18.89%, test accuracy = 18.04%
Epoch = 8, train accuracy = 19.08%, test accuracy = 17.98%
Epoch = 9, train accuracy = 19.23%, test accuracy = 18.00%
Epoch = 10, train accuracy = 19.41%, test accuracy = 17.82%
Epoch = 11, train accuracy = 19.52%, test accuracy = 17.88%
Epoch = 12, train accuracy = 19.73%, test accuracy = 17.92%
Epoch = 13, train accuracy = 19.94%, test accuracy = 17.95%
Epoch = 14, train accuracy = 20.12%, test accuracy = 17.74%
Epoch = 15, train accuracy = 20.22%, test accuracy = 17.86%
Epoch = 16, train accuracy = 20.29%, test accuracy = 17.86%
Epoch = 17, train accuracy = 20.42%, test accurac

Epoch = 138, train accuracy = 25.84%, test accuracy = 18.47%
Epoch = 139, train accuracy = 25.94%, test accuracy = 18.60%
Epoch = 140, train accuracy = 25.91%, test accuracy = 18.58%
Epoch = 141, train accuracy = 26.00%, test accuracy = 18.52%
Epoch = 142, train accuracy = 25.95%, test accuracy = 18.38%
Epoch = 143, train accuracy = 26.01%, test accuracy = 18.44%
Epoch = 144, train accuracy = 26.03%, test accuracy = 18.45%
Epoch = 145, train accuracy = 26.08%, test accuracy = 18.56%
Epoch = 146, train accuracy = 26.05%, test accuracy = 18.54%
Epoch = 147, train accuracy = 26.08%, test accuracy = 18.40%
Epoch = 148, train accuracy = 26.07%, test accuracy = 18.35%
Epoch = 149, train accuracy = 26.10%, test accuracy = 18.39%
Epoch = 150, train accuracy = 26.11%, test accuracy = 18.44%
Epoch = 151, train accuracy = 26.16%, test accuracy = 18.54%
Epoch = 152, train accuracy = 26.15%, test accuracy = 18.55%
Epoch = 153, train accuracy = 26.19%, test accuracy = 18.52%
Epoch = 154, train accur

Epoch = 273, train accuracy = 28.90%, test accuracy = 19.00%
Epoch = 274, train accuracy = 28.86%, test accuracy = 18.99%
Epoch = 275, train accuracy = 28.89%, test accuracy = 18.98%
Epoch = 276, train accuracy = 28.99%, test accuracy = 18.97%
Epoch = 277, train accuracy = 28.99%, test accuracy = 19.01%
Epoch = 278, train accuracy = 28.87%, test accuracy = 19.08%
Epoch = 279, train accuracy = 28.99%, test accuracy = 19.01%
Epoch = 280, train accuracy = 28.96%, test accuracy = 19.00%
Epoch = 281, train accuracy = 29.07%, test accuracy = 19.00%
Epoch = 282, train accuracy = 29.06%, test accuracy = 18.99%
Epoch = 283, train accuracy = 29.07%, test accuracy = 19.04%
Epoch = 284, train accuracy = 29.09%, test accuracy = 18.97%
Epoch = 285, train accuracy = 29.08%, test accuracy = 19.01%
Epoch = 286, train accuracy = 29.12%, test accuracy = 18.92%
Epoch = 287, train accuracy = 29.05%, test accuracy = 19.08%
Epoch = 288, train accuracy = 29.17%, test accuracy = 19.01%
Epoch = 289, train accur

Epoch = 408, train accuracy = 31.18%, test accuracy = 19.12%
Epoch = 409, train accuracy = 31.21%, test accuracy = 19.14%
Epoch = 410, train accuracy = 31.22%, test accuracy = 19.29%
Epoch = 411, train accuracy = 31.24%, test accuracy = 19.23%
Epoch = 412, train accuracy = 31.22%, test accuracy = 19.12%
Epoch = 413, train accuracy = 31.28%, test accuracy = 19.16%
Epoch = 414, train accuracy = 31.31%, test accuracy = 19.25%
Epoch = 415, train accuracy = 31.23%, test accuracy = 19.24%
Epoch = 416, train accuracy = 31.31%, test accuracy = 19.23%
Epoch = 417, train accuracy = 31.28%, test accuracy = 19.20%
Epoch = 418, train accuracy = 31.33%, test accuracy = 19.21%
Epoch = 419, train accuracy = 31.36%, test accuracy = 19.31%
Epoch = 420, train accuracy = 31.30%, test accuracy = 19.23%
Epoch = 421, train accuracy = 31.32%, test accuracy = 19.12%
Epoch = 422, train accuracy = 31.35%, test accuracy = 19.17%
Epoch = 423, train accuracy = 31.29%, test accuracy = 19.11%
Epoch = 424, train accur

Epoch = 543, train accuracy = 33.02%, test accuracy = 19.49%
Epoch = 544, train accuracy = 33.03%, test accuracy = 19.47%
Epoch = 545, train accuracy = 33.00%, test accuracy = 19.46%
Epoch = 546, train accuracy = 33.00%, test accuracy = 19.49%
Epoch = 547, train accuracy = 33.00%, test accuracy = 19.51%
Epoch = 548, train accuracy = 33.04%, test accuracy = 19.42%
Epoch = 549, train accuracy = 33.04%, test accuracy = 19.42%
Epoch = 550, train accuracy = 33.05%, test accuracy = 19.39%
Epoch = 551, train accuracy = 33.09%, test accuracy = 19.37%
Epoch = 552, train accuracy = 33.08%, test accuracy = 19.45%
Epoch = 553, train accuracy = 33.07%, test accuracy = 19.45%
Epoch = 554, train accuracy = 33.17%, test accuracy = 19.48%
Epoch = 555, train accuracy = 33.17%, test accuracy = 19.44%
Epoch = 556, train accuracy = 33.08%, test accuracy = 19.41%
Epoch = 557, train accuracy = 33.12%, test accuracy = 19.40%
Epoch = 558, train accuracy = 33.21%, test accuracy = 19.50%
Epoch = 559, train accur

Epoch = 678, train accuracy = 34.58%, test accuracy = 19.25%
Epoch = 679, train accuracy = 34.54%, test accuracy = 19.36%
Epoch = 680, train accuracy = 34.57%, test accuracy = 19.16%
Epoch = 681, train accuracy = 34.61%, test accuracy = 19.20%
Epoch = 682, train accuracy = 34.56%, test accuracy = 19.21%
Epoch = 683, train accuracy = 34.52%, test accuracy = 19.37%
Epoch = 684, train accuracy = 34.72%, test accuracy = 19.28%
Epoch = 685, train accuracy = 34.67%, test accuracy = 19.38%
Epoch = 686, train accuracy = 34.61%, test accuracy = 19.29%
Epoch = 687, train accuracy = 34.66%, test accuracy = 19.17%
Epoch = 688, train accuracy = 34.77%, test accuracy = 19.22%
Epoch = 689, train accuracy = 34.71%, test accuracy = 19.24%
Epoch = 690, train accuracy = 34.77%, test accuracy = 19.29%
Epoch = 691, train accuracy = 34.72%, test accuracy = 19.12%
Epoch = 692, train accuracy = 34.71%, test accuracy = 19.29%
Epoch = 693, train accuracy = 34.80%, test accuracy = 19.14%
Epoch = 694, train accur

Epoch = 813, train accuracy = 36.01%, test accuracy = 19.28%
Epoch = 814, train accuracy = 36.01%, test accuracy = 19.38%
Epoch = 815, train accuracy = 35.91%, test accuracy = 19.29%
Epoch = 816, train accuracy = 36.03%, test accuracy = 19.39%
Epoch = 817, train accuracy = 36.01%, test accuracy = 19.29%
Epoch = 818, train accuracy = 35.97%, test accuracy = 19.32%
Epoch = 819, train accuracy = 36.08%, test accuracy = 19.29%
Epoch = 820, train accuracy = 35.97%, test accuracy = 19.28%
Epoch = 821, train accuracy = 35.99%, test accuracy = 19.15%
Epoch = 822, train accuracy = 36.01%, test accuracy = 19.32%
Epoch = 823, train accuracy = 36.02%, test accuracy = 19.44%
Epoch = 824, train accuracy = 36.12%, test accuracy = 19.15%
Epoch = 825, train accuracy = 36.04%, test accuracy = 19.26%
Epoch = 826, train accuracy = 36.09%, test accuracy = 19.21%
Epoch = 827, train accuracy = 36.09%, test accuracy = 19.42%
Epoch = 828, train accuracy = 36.12%, test accuracy = 19.36%
Epoch = 829, train accur

Epoch = 948, train accuracy = 37.07%, test accuracy = 19.55%
Epoch = 949, train accuracy = 37.20%, test accuracy = 19.52%
Epoch = 950, train accuracy = 37.09%, test accuracy = 19.48%
Epoch = 951, train accuracy = 37.21%, test accuracy = 19.63%
Epoch = 952, train accuracy = 37.24%, test accuracy = 19.67%
Epoch = 953, train accuracy = 37.21%, test accuracy = 19.55%
Epoch = 954, train accuracy = 37.19%, test accuracy = 19.65%
Epoch = 955, train accuracy = 37.29%, test accuracy = 19.44%
Epoch = 956, train accuracy = 37.36%, test accuracy = 19.48%
Epoch = 957, train accuracy = 37.32%, test accuracy = 19.66%
Epoch = 958, train accuracy = 37.40%, test accuracy = 19.38%
Epoch = 959, train accuracy = 37.41%, test accuracy = 19.47%
Epoch = 960, train accuracy = 37.32%, test accuracy = 19.68%
Epoch = 961, train accuracy = 37.35%, test accuracy = 19.45%
Epoch = 962, train accuracy = 37.27%, test accuracy = 19.64%
Epoch = 963, train accuracy = 37.37%, test accuracy = 19.52%
Epoch = 964, train accur

In [32]:
import tensorflow as tf
tf.reset_default_graph()
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.model_selection import train_test_split

def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

def get_data():
    return train_test_split(mel_spectral_matrix1, output, test_size=0.33)

def main():
    train_X, test_X, train_y, test_y = get_data()
    # Turn the targets to categorical
    train_y = tf.keras.utils.to_categorical(train_y)
    test_y = tf.keras.utils.to_categorical(test_y)
    # Layer's sizes
    x_size = train_X.shape[1]
    h_size = 256    
    y_size = train_y.shape[1]
    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])
    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.AdamOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    print("Confusion Matrix:")
    conf_mat = confusion_matrix(np.argmax(test_y, axis=1), 
                 sess.run(predict, feed_dict={X: test_X, y: test_y}))
    print(conf_mat)
    
    sess.close()

if __name__ == '__main__':
    main()

Confusion Matrix:
[[   0    0    0 1001    0    0    0    0    0    0]
 [   0    0    0 1013    0    0    0    0    0    0]
 [   0    0    1  978    0    0    0    0    0    0]
 [   0    0    1  984    0    1    0    0    0    0]
 [   0    0    0  970    0    0    0    0    0    0]
 [   0    0   13  935    0    8    6    1    0   13]
 [   0    0    0  979    0    0    0    0    0    0]
 [   0    0    1  977    0    1    0    0    0    1]
 [   0    0    0 1014    0    0    0    0    0    1]
 [   0    0    0 1005    0    0    0    0    0    0]]


In [11]:
import tensorflow as tf
tf.reset_default_graph()
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.model_selection import train_test_split

def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

def get_data():
    return train_test_split(mel_spectral_matrix1, output, test_size=0.33)

def main():
    train_X, test_X, train_y, test_y = get_data()
    # Turn the targets to categorical
    train_y = tf.keras.utils.to_categorical(train_y)
    test_y = tf.keras.utils.to_categorical(test_y)
    # Layer's sizes
    x_size = train_X.shape[1]
    h_size = 256    
    y_size = train_y.shape[1]
    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])
    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(1000):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})
        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: train_X, y: train_y}))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: test_X, y: test_y}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
    
    sess.close()

if __name__ == '__main__':
    main()

Epoch = 1, train accuracy = 12.01%, test accuracy = 11.99%
Epoch = 2, train accuracy = 12.26%, test accuracy = 12.18%
Epoch = 3, train accuracy = 12.35%, test accuracy = 12.27%
Epoch = 4, train accuracy = 12.47%, test accuracy = 12.37%
Epoch = 5, train accuracy = 12.51%, test accuracy = 12.41%
Epoch = 6, train accuracy = 12.56%, test accuracy = 12.47%
Epoch = 7, train accuracy = 12.60%, test accuracy = 12.55%
Epoch = 8, train accuracy = 12.67%, test accuracy = 12.57%
Epoch = 9, train accuracy = 12.69%, test accuracy = 12.57%
Epoch = 10, train accuracy = 12.73%, test accuracy = 12.57%
Epoch = 11, train accuracy = 12.77%, test accuracy = 12.59%
Epoch = 12, train accuracy = 12.83%, test accuracy = 12.64%
Epoch = 13, train accuracy = 12.87%, test accuracy = 12.67%
Epoch = 14, train accuracy = 12.92%, test accuracy = 12.70%
Epoch = 15, train accuracy = 12.97%, test accuracy = 12.67%
Epoch = 16, train accuracy = 13.04%, test accuracy = 12.70%
Epoch = 17, train accuracy = 13.11%, test accurac

Epoch = 138, train accuracy = 15.42%, test accuracy = 12.98%
Epoch = 139, train accuracy = 15.44%, test accuracy = 13.00%
Epoch = 140, train accuracy = 15.45%, test accuracy = 13.00%
Epoch = 141, train accuracy = 15.45%, test accuracy = 13.01%
Epoch = 142, train accuracy = 15.48%, test accuracy = 12.98%
Epoch = 143, train accuracy = 15.50%, test accuracy = 12.98%
Epoch = 144, train accuracy = 15.53%, test accuracy = 12.98%
Epoch = 145, train accuracy = 15.53%, test accuracy = 13.00%
Epoch = 146, train accuracy = 15.53%, test accuracy = 13.03%
Epoch = 147, train accuracy = 15.55%, test accuracy = 13.03%
Epoch = 148, train accuracy = 15.56%, test accuracy = 13.03%
Epoch = 149, train accuracy = 15.58%, test accuracy = 13.01%
Epoch = 150, train accuracy = 15.60%, test accuracy = 13.01%
Epoch = 151, train accuracy = 15.62%, test accuracy = 13.04%
Epoch = 152, train accuracy = 15.62%, test accuracy = 13.04%
Epoch = 153, train accuracy = 15.63%, test accuracy = 13.01%
Epoch = 154, train accur

Epoch = 273, train accuracy = 16.74%, test accuracy = 13.25%
Epoch = 274, train accuracy = 16.74%, test accuracy = 13.26%
Epoch = 275, train accuracy = 16.75%, test accuracy = 13.27%
Epoch = 276, train accuracy = 16.75%, test accuracy = 13.27%
Epoch = 277, train accuracy = 16.75%, test accuracy = 13.29%
Epoch = 278, train accuracy = 16.74%, test accuracy = 13.28%
Epoch = 279, train accuracy = 16.74%, test accuracy = 13.28%
Epoch = 280, train accuracy = 16.73%, test accuracy = 13.26%
Epoch = 281, train accuracy = 16.72%, test accuracy = 13.26%
Epoch = 282, train accuracy = 16.72%, test accuracy = 13.26%
Epoch = 283, train accuracy = 16.72%, test accuracy = 13.26%
Epoch = 284, train accuracy = 16.72%, test accuracy = 13.23%
Epoch = 285, train accuracy = 16.72%, test accuracy = 13.24%
Epoch = 286, train accuracy = 16.73%, test accuracy = 13.23%
Epoch = 287, train accuracy = 16.75%, test accuracy = 13.23%
Epoch = 288, train accuracy = 16.76%, test accuracy = 13.22%
Epoch = 289, train accur

Epoch = 408, train accuracy = 17.34%, test accuracy = 13.21%
Epoch = 409, train accuracy = 17.35%, test accuracy = 13.22%
Epoch = 410, train accuracy = 17.35%, test accuracy = 13.22%
Epoch = 411, train accuracy = 17.35%, test accuracy = 13.22%
Epoch = 412, train accuracy = 17.35%, test accuracy = 13.22%
Epoch = 413, train accuracy = 17.35%, test accuracy = 13.22%
Epoch = 414, train accuracy = 17.36%, test accuracy = 13.23%
Epoch = 415, train accuracy = 17.36%, test accuracy = 13.23%
Epoch = 416, train accuracy = 17.36%, test accuracy = 13.23%
Epoch = 417, train accuracy = 17.37%, test accuracy = 13.24%
Epoch = 418, train accuracy = 17.38%, test accuracy = 13.24%
Epoch = 419, train accuracy = 17.38%, test accuracy = 13.23%
Epoch = 420, train accuracy = 17.38%, test accuracy = 13.23%
Epoch = 421, train accuracy = 17.40%, test accuracy = 13.21%
Epoch = 422, train accuracy = 17.41%, test accuracy = 13.21%
Epoch = 423, train accuracy = 17.42%, test accuracy = 13.22%
Epoch = 424, train accur

Epoch = 543, train accuracy = 17.93%, test accuracy = 13.34%
Epoch = 544, train accuracy = 17.94%, test accuracy = 13.33%
Epoch = 545, train accuracy = 17.95%, test accuracy = 13.32%
Epoch = 546, train accuracy = 17.96%, test accuracy = 13.33%
Epoch = 547, train accuracy = 17.96%, test accuracy = 13.32%
Epoch = 548, train accuracy = 17.96%, test accuracy = 13.32%
Epoch = 549, train accuracy = 17.97%, test accuracy = 13.32%
Epoch = 550, train accuracy = 17.97%, test accuracy = 13.32%
Epoch = 551, train accuracy = 17.97%, test accuracy = 13.32%
Epoch = 552, train accuracy = 17.99%, test accuracy = 13.33%
Epoch = 553, train accuracy = 17.99%, test accuracy = 13.34%
Epoch = 554, train accuracy = 17.99%, test accuracy = 13.34%
Epoch = 555, train accuracy = 18.01%, test accuracy = 13.33%
Epoch = 556, train accuracy = 18.01%, test accuracy = 13.30%
Epoch = 557, train accuracy = 18.01%, test accuracy = 13.31%
Epoch = 558, train accuracy = 18.01%, test accuracy = 13.32%
Epoch = 559, train accur

Epoch = 678, train accuracy = 18.56%, test accuracy = 13.45%
Epoch = 679, train accuracy = 18.56%, test accuracy = 13.45%
Epoch = 680, train accuracy = 18.56%, test accuracy = 13.45%
Epoch = 681, train accuracy = 18.56%, test accuracy = 13.46%
Epoch = 682, train accuracy = 18.58%, test accuracy = 13.47%
Epoch = 683, train accuracy = 18.59%, test accuracy = 13.47%
Epoch = 684, train accuracy = 18.59%, test accuracy = 13.47%
Epoch = 685, train accuracy = 18.59%, test accuracy = 13.47%
Epoch = 686, train accuracy = 18.59%, test accuracy = 13.48%
Epoch = 687, train accuracy = 18.60%, test accuracy = 13.47%
Epoch = 688, train accuracy = 18.60%, test accuracy = 13.47%
Epoch = 689, train accuracy = 18.62%, test accuracy = 13.47%
Epoch = 690, train accuracy = 18.63%, test accuracy = 13.48%
Epoch = 691, train accuracy = 18.64%, test accuracy = 13.48%
Epoch = 692, train accuracy = 18.65%, test accuracy = 13.49%
Epoch = 693, train accuracy = 18.67%, test accuracy = 13.48%
Epoch = 694, train accur

Epoch = 813, train accuracy = 19.13%, test accuracy = 13.39%
Epoch = 814, train accuracy = 19.13%, test accuracy = 13.40%
Epoch = 815, train accuracy = 19.14%, test accuracy = 13.40%
Epoch = 816, train accuracy = 19.15%, test accuracy = 13.40%
Epoch = 817, train accuracy = 19.15%, test accuracy = 13.40%
Epoch = 818, train accuracy = 19.14%, test accuracy = 13.40%
Epoch = 819, train accuracy = 19.14%, test accuracy = 13.40%
Epoch = 820, train accuracy = 19.14%, test accuracy = 13.41%
Epoch = 821, train accuracy = 19.14%, test accuracy = 13.40%
Epoch = 822, train accuracy = 19.13%, test accuracy = 13.40%
Epoch = 823, train accuracy = 19.14%, test accuracy = 13.41%
Epoch = 824, train accuracy = 19.14%, test accuracy = 13.41%
Epoch = 825, train accuracy = 19.14%, test accuracy = 13.41%
Epoch = 826, train accuracy = 19.16%, test accuracy = 13.39%
Epoch = 827, train accuracy = 19.16%, test accuracy = 13.39%
Epoch = 828, train accuracy = 19.16%, test accuracy = 13.39%
Epoch = 829, train accur

Epoch = 948, train accuracy = 19.61%, test accuracy = 13.34%
Epoch = 949, train accuracy = 19.61%, test accuracy = 13.34%
Epoch = 950, train accuracy = 19.61%, test accuracy = 13.34%
Epoch = 951, train accuracy = 19.61%, test accuracy = 13.34%
Epoch = 952, train accuracy = 19.61%, test accuracy = 13.35%
Epoch = 953, train accuracy = 19.61%, test accuracy = 13.35%
Epoch = 954, train accuracy = 19.61%, test accuracy = 13.35%
Epoch = 955, train accuracy = 19.61%, test accuracy = 13.34%
Epoch = 956, train accuracy = 19.62%, test accuracy = 13.35%
Epoch = 957, train accuracy = 19.62%, test accuracy = 13.38%
Epoch = 958, train accuracy = 19.62%, test accuracy = 13.37%
Epoch = 959, train accuracy = 19.63%, test accuracy = 13.36%
Epoch = 960, train accuracy = 19.64%, test accuracy = 13.35%
Epoch = 961, train accuracy = 19.65%, test accuracy = 13.35%
Epoch = 962, train accuracy = 19.65%, test accuracy = 13.35%
Epoch = 963, train accuracy = 19.65%, test accuracy = 13.35%
Epoch = 964, train accur

In [23]:
import tensorflow as tf
tf.reset_default_graph()
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.model_selection import train_test_split

def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

def get_data():
    return train_test_split(mel_spectral_matrix1, output, test_size=0.33)

def main():
    train_X, test_X, train_y, test_y = get_data()
    # Turn the targets to categorical
    train_y = tf.keras.utils.to_categorical(train_y)
    test_y = tf.keras.utils.to_categorical(test_y)
    # Layer's sizes
    x_size = train_X.shape[1]
    h_size = 256    
    y_size = train_y.shape[1]
    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])
    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    print("Confusion Matrix:")
    conf_mat = confusion_matrix(np.argmax(test_y, axis=1), 
                 sess.run(predict, feed_dict={X: test_X, y: test_y}))
    print(conf_mat)
    
    sess.close()

if __name__ == '__main__':
    main()

Confusion Matrix:
[[   0    0    0    0    0    0    0    0  982    0]
 [   0    0    0    0    0    0    0    0  972    0]
 [   0    0    0    1    0    0    0    0  970    0]
 [   0    1    0    1    0    0    0    0  981    0]
 [   0    0    0    0    0    0    0    0  967    0]
 [   8    5    0   13    0    1    2    2  990    0]
 [   0    0    0    0    0    0    0    0 1015    0]
 [   1    2    0    3    0    0    0    0  995    0]
 [   0    0    0    0    0    0    0    1  993    0]
 [   0    0    0    0    0    0    0    0  998    0]]


# Clustering

In [92]:
X = mel_spectral_matrix1.copy()
print(X)
y = output.copy()

[[1.03333504e-03 2.75104954e-04 3.21582523e-04 ... 7.67784797e-05
  9.52182932e-05 8.12144487e-05]
 [7.13783561e-05 3.76327029e-05 1.14335002e-04 ... 7.80411522e-06
  1.82020278e-06 4.50857785e-06]
 [1.50158098e-05 7.81936518e-05 3.11857675e-04 ... 1.17386418e-04
  8.23624399e-05 1.49524310e-04]
 ...
 [7.07971531e-16 7.51806050e-15 9.05824445e-17 ... 7.22920846e-18
  3.83486976e-17 4.97453376e-17]
 [8.57995875e-17 4.02120008e-17 1.18203358e-16 ... 3.88365842e-18
  6.00108299e-18 1.86017357e-17]
 [8.23904654e-17 6.97276859e-19 9.74128400e-18 ... 3.93743855e-17
  4.63124364e-09 6.77912567e-08]]


In [77]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
kmean = KMeans(10)
kmean.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [80]:
predicted = kmean.fit_predict(X)
predicted

array([0, 0, 0, ..., 0, 0, 0])

Since our input matrix (A) is 2 dimensional and not 1 dimensional, we are not able to get an accurate representation of our k mean clustering. We get only 1 cluster. In addition, when making the input matrix (A) into 1 dimensional, we get the same result of having 1 cluster and 3001 points in that 1 cluster. This is because we have the values are close to each other and significantly small, we are not able to form much of a cluster since each point would be so close to one another. As a result, with the values in the training matrix A, we could not separate these values into 10 clusters.